# VTC_DataAnalytics

I will tackle real-life questions pertaining to VTC Data Analytics in my preferred format.

When something is unclear, I will state clearly my assumptions and will go ahead as I see fit, inlcuding all elements.

I will have fun in the process - see what I can come up with.

In [1]:
# import libraries
import sqlite3
#
import pandas as pd

# Part 1

Create a database importing data from Excel.

We do this to simulate how to write an executable SQL query to answer the question.

We can do this with SQLite, which is a self-contained, serverless, zero-configuration, transactional SQL database engine.

In [2]:
conn = sqlite3.connect('VTC_DataAnalytics.db') #SQLite database file
df = pd.read_excel('mobile_events.xlsx') #path to the Excel file dataset
df.to_sql('MOBILE_EVENTS', conn, if_exists='replace', index=False) #a table within the database


65534

Define a single SQL query, assuming read-only access to the database (i.e. do not use ``CREATE TABLE``)

What is the percentage of sessions which end up in a ride order everyday?

Assumptions:
1) Session definition: a Session is a product life-cycle journey by ``device_id`` that starts with "pickup_shown" and finishes with "ride_order_tapped".\
Note: a Sesion in Google Analytics within the context of mobile apps is a journey since the user opens the App and closes it. This has to not be confused with a Session within our VTC context.

2) We will be agnostic as to what happens within the session, and only measure sessions with a "pickup_shown" start and a "ride_order_tapped" finish. The user can go back and forth during the pre-order session, so we only count the first "pickup_shown".

3) We need to accommodate for the possibility of the same ``device_id`` having multiple sessions during the day.

4) We need to accommodate for the possibility of a session to start before the end of day theshold, in which case we will use the start of the session ("pickup_shown") as the timestamp that defines the daily session that can potentially end up in a ride order.

In [3]:
# query = """
# SELECT device_id, timestamp, event_name, DATE(timestamp) as date from MOBILE_EVENTS
# where event_name = "pickup_shown" or event_name = "ride_order_tapped"
# """

query = """
Select *
from MOBILE_EVENTS
where event_name = "pickup_shown" or event_name = "ride_order_tapped"
"""

# Execute the query
pd.read_sql_query(query, conn)#.to_csv('test.csv')

,device_id,event_name,timestamp
0,1,pickup_shown,2020-10-02 15:36:02
1,2,pickup_shown,2020-10-06 17:05:31
2,2,pickup_shown,2020-10-06 17:06:10
3,2,pickup_shown,2020-10-06 17:06:14
4,2,pickup_shown,2020-10-06 17:07:35
...,...,...,...
21457,865,pickup_shown,2020-10-03 09:26:50
21458,865,pickup_shown,2020-10-03 09:26:51
21459,865,pickup_shown,2020-10-03 10:07:21
21460,865,pickup_shown,2020-10-03 10:41:08


In [4]:
# close connection:
conn.close()